In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('Crawled_data.db')

In [3]:
cursor = conn.cursor()

In [4]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
import json
import hashlib
import random
import signal
import sys
from threading import Thread, Lock, current_thread
from queue import Queue
from datetime import datetime
import pandas as pd

# Initialize a set to store crawled URLs and a lock for thread-safe operations
crawled_urls = set()
crawled_urls_lock = Lock()

# Initialize a dictionary to store visited URLs and their hash values
visited_urls = {}
visited_urls_lock = Lock()

# Initialize a queue for URLs to be crawled
url_queue = Queue()

# Initialize a DataFrame to store visited URLs and their hash values
data_frame_1 = pd.DataFrame(columns=['url', 'hash'])

# List of User-Agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1'
]

# Function to fetch page content
def fetch_page(session, url):
    """Fetch the content of the page at the given URL."""
    headers = {
        'User-Agent': random.choice(user_agents)
    }
    try:
        response = session.get(url, headers=headers)
        if response.status_code == 200 and response.text.strip():  # Check for non-empty content
            return response.text
        else:
            print(f"Non-200 status code or empty content {response.status_code} for {url}")
            return None
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

# Function to extract links from HTML content
def get_links(html_content, base_url):
    """Extract and return all the links from the HTML content."""
    soup = BeautifulSoup(html_content, 'html.parser')
    links = set()
    for anchor in soup.find_all('a', href=True):
        href = anchor['href']
        full_url = urljoin(base_url, href)
        if urlparse(full_url).netloc == urlparse(base_url).netloc:
            links.add(full_url)
    return links

# Signal handler for graceful shutdown
def signal_handler(sig, frame):
    print('Interrupted! Saving checkpoint and visited URLs to CSV...')
    save_checkpoint()
    save_visited_urls_csv()
    sys.exit(0)

# Register the signal handler
signal.signal(signal.SIGINT, signal_handler)

# Function to load the existing state from the checkpoint file
def load_checkpoint():
    """Load the existing state of visited URLs from the checkpoint file (master.csv)."""
    global visited_urls, data_frame_1
    try:
        df = pd.read_csv('master.csv')
        with visited_urls_lock:
            visited_urls.update(df.set_index('url').to_dict()['hash'])
        data_frame_1 = df.copy()
        print("Checkpoint loaded.")
    except FileNotFoundError:
        print("No checkpoint found. Starting fresh.")
    except Exception as e:
        print(f"Error loading checkpoint file: {e}. Starting fresh.")

# Function to save the current state to a checkpoint file
def save_checkpoint():
    """Save the current state of visited URLs to a checkpoint file (master.csv)."""
    global data_frame_1
    with visited_urls_lock:
        data_frame_1 = pd.DataFrame(list(visited_urls.items()), columns=['url', 'hash'])
        data_frame_1.to_csv('master.csv', index=False)
    print("Checkpoint saved.")

# Function to save visited URLs to a CSV file with current date and time in filename
def save_visited_urls_csv():
    """Save visited URLs to a CSV file with current date and time in filename."""
    global data_frame_1
    current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'visited_urls_{current_datetime}.csv'
    with visited_urls_lock:
        data_frame_1.to_csv(filename, index=False)
    print(f"Visited URLs saved to {filename}")

# Worker function for crawling with uniqueness check
def worker(session, max_depth, delay):
    while True:
        current_url, depth = url_queue.get()
        if current_url is None:
            break

        if depth > max_depth:
            url_queue.task_done()
            continue

        print(f"[{current_thread().name}] Fetching {current_url}")
        html_content = fetch_page(session, current_url)
        time.sleep(delay)  # Delay between requests

        if html_content:
            url_hash = hashlib.sha256(current_url.encode()).hexdigest()
            with crawled_urls_lock:
                if current_url not in crawled_urls:
                    crawled_urls.add(current_url)

            with visited_urls_lock:
                if current_url not in visited_urls:
                    visited_urls[current_url] = url_hash

            if depth < max_depth:
                links = get_links(html_content, current_url)
                for link in links:
                    link_hash = hashlib.sha256(link.encode()).hexdigest()
                    with crawled_urls_lock:
                        if link not in crawled_urls:
                            crawled_urls.add(link)
                            url_queue.put((link, depth + 1))
                            with visited_urls_lock:
                                if link not in visited_urls:
                                    visited_urls[link] = link_hash

        else:
            print(f"[{current_thread().name}] Failed to retrieve {current_url}")

        url_queue.task_done()

# Function to crawl the website with uniqueness check
def crawl(url, max_depth, delay, num_threads):
    """Crawl the website starting from the given URL up to a maximum depth."""
    session = requests.Session()

    # Load the checkpoint if it exists
    load_checkpoint()

    if url not in crawled_urls:
        url_queue.put((url, 0))

    threads = []
    for i in range(num_threads):
        thread = Thread(target=worker, args=(session, max_depth, delay), name=f"Thread-{i+1}")
        thread.start()
        threads.append(thread)

    # Wait for the queue to be empty
    url_queue.join()

    # Stop the threads
    for _ in range(num_threads):
        url_queue.put((None, 0))
    for thread in threads:
        thread.join()

    # After crawling is complete, save the checkpoint and visited URLs to CSV
    save_checkpoint()
    save_visited_urls_csv()

# Define parameters
start_url = 'https://www.republicworld.com/'
max_depth = 5  # Adjust the maximum depth of crawling as needed
delay = 1  # 1-second delay between requests to be polite to the server
num_threads = 16  # Number of threads for multithreading

# Start crawling
crawl(start_url, max_depth, delay, num_threads)




Checkpoint loaded.
[Thread-1] Fetching https://www.republicworld.com/
[Thread-1] Fetching https://www.republicworld.com/sports/cricket/world-cup/team-india-returns-live-t20-world-cup-champions-rohit-sharma-virat-kohli-arrive-in-new-delhi-meet-pm-modi-mumbai-parade-latest-updates[Thread-2] Fetching https://www.republicworld.com/automobile
[Thread-4] Fetching https://www.republicworld.com/web-stories
[Thread-7] Fetching https://www.republicworld.com/entertainment/telugu-cinema/pawan-kalyan-s-og-ustaad-bhagat-singh-shelved-midway-after-powerstar-becomes-deputy-cm-of-andhra
[Thread-10] Fetching https://www.republicworld.com/lifestyle/fashion/king-of-rock-and-roll-elvis-presley-s-legendary-blue-suede-shoes-auctioned-at-a-shocking-150000
[Thread-11] Fetching https://www.republicworld.com/entertainment/bollywood/kalki-2898-ad-hindi-box-office-collection-week-1-prabhas-starrer-surpasses-150-crore-mark
[Thread-13] Fetching https://www.republicworld.com/web-stories/lifestyle/bored-of-sourdough-b

In [5]:
# The DataFrame 'data_frame_1' contains the visited URLs and their hash values
data_frame_1

,url,hash
0,https://www.republicworld.com/,189283961cc61cfcdbb8f03a062d6531f0c86846d981b7...
1,https://www.republicworld.com/shorts/entertain...,402c71d580651e357ce593287787a38c8ecb038b31a66c...
2,https://www.republicworld.com/automobile/auto-...,4fcdf21dcce738ac92831d47287c9fa7d1832a15fcd100...
3,https://www.republicworld.com/education,0d44150d4d5c43f8d45fd527d2d7bae8e44ebbe4ff58fa...
4,https://www.republicworld.com/lifestyle/food/f...,b13fa95b963ea0980022fc9c0c5b95930682311a3fa198...
...,...,...
3651,https://www.republicworld.com/india/amit-shah-...,0a3e8bbd830be9833f73e1cf56e06cccc4e7809f5eb35a...
3652,https://www.republicworld.com/india/reasi-terr...,010a149752dd62ffb4116593cd972905f6965bb1c39ee4...
3653,https://www.republicworld.com/india/live-updat...,d2fbd87a467cdf9aabcceb651e6e0f6b97e3120be57682...
3654,https://www.republicworld.com/india/india-news...,53239348fe7de1a52fbe4e4a7bbd23957435285603a308...


In [6]:
table_name = 'master'
data_frame_1.to_sql(table_name, conn, if_exists='replace', index=False)

3656

In [7]:
cursor.execute("SELECT * FROM master")

# Fetch all rows from the result set
rows = cursor.fetchall()

# Print each row
for row in rows:
    print(row)

('https://www.republicworld.com/', '189283961cc61cfcdbb8f03a062d6531f0c86846d981b7862f080694b2e70076')
('https://www.republicworld.com/shorts/entertainment/sonakshi-sinha-zaheer-s-wedding-varun-sharma-sanjay-bhansali-and-others', '402c71d580651e357ce593287787a38c8ecb038b31a66cb36a9e9c7c959469b0')
('https://www.republicworld.com/automobile/auto-sales-mm-sold-40022-suvs-bajaj-auto-posts-5-growth-in-june', '4fcdf21dcce738ac92831d47287c9fa7d1832a15fcd100d2175ba7a28e20f953')
('https://www.republicworld.com/education', '0d44150d4d5c43f8d45fd527d2d7bae8e44ebbe4ff58fa1c5bbe38605705b522')
('https://www.republicworld.com/lifestyle/food/five-essential-superfoods-to-keep-working-women-energised', 'b13fa95b963ea0980022fc9c0c5b95930682311a3fa19895e95ec1d3306dbd4c')
('https://www.republicworld.com/latest-news', '44af9447b807719a5602180e402bef5a668b8e20dceb465909a9820da96765bf')
('https://www.republicworld.com/economy', '65277608e0da80e9b582e12030d7bf68fd8b8afc6517ebb9f62f76c2774ca768')
('https://www.

In [11]:
import os

# Step 1: Create Scrapy Project
if not os.path.exists('mycrawler'):
    os.system('scrapy startproject mycrawler')

In [12]:
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from twisted.internet import reactor, defer
from bs4 import BeautifulSoup
import json
from datetime import datetime
import pandas as pd
import os
import re
import sqlite3

# Variable to hold the HTML content for each URL
html_contents = {}

# Define the state file path
state_file_path = 'scraping_state.json'

# Function to save the state
def save_state():
    global html_contents
    state = {
        'html_contents': html_contents
    }
    with open(state_file_path, 'w') as f:
        json.dump(state, f)

# Function to load the state
def load_state():
    global html_contents
    if os.path.exists(state_file_path):
        with open(state_file_path, 'r') as f:
            state = json.load(f)
        html_contents = state.get('html_contents', {})
    else:
        html_contents.clear()

# Define the Spider class
class HTMLSpider(scrapy.Spider):
    name = "html_spider"
    rotate_user_agent = True  # Rotate user-agent for each request

    def __init__(self, url=None, *args, **kwargs):
        super(HTMLSpider, self).__init__(*args, **kwargs)
        self.start_urls = [url]

    def parse(self, response):
        global html_contents
        html_contents[response.url] = response.body.decode('utf-8')
        self.log(f'Fetched HTML content from {response.url}')
        
        # Save the state after fetching each URL
        save_state()

# Function to run the spider
def run_spiders(urls_to_process):
    runner = CrawlerRunner(get_project_settings())

    @defer.inlineCallbacks
    def crawl():
        for url in urls_to_process:
            yield runner.crawl(HTMLSpider, url=url)
        reactor.stop()

    crawl()
    reactor.run()

# Function to extract publication date from the provided HTML
def extract_pub_date(html):
    soup = BeautifulSoup(html, 'html.parser')

    # Try extracting from meta tags
    pub_date = soup.find('meta', {'name': 'pubdate'}) or soup.find('meta', {'property': 'article:published_time'}) or soup.find('meta', {'property': 'og:article:published_time'})
    if pub_date and pub_date.get('content'):
        return pub_date['content']

    # Try extracting from structured data
    json_ld = soup.find('script', {'type': 'application/ld+json'})
    if json_ld:
        data = json.loads(json_ld.string)
        if isinstance(data, list):  # Sometimes JSON-LD is an array of objects
            for item in data:
                if 'datePublished' in item:
                    return item['datePublished']
        else:
            if 'datePublished' in data:
                return data['datePublished']

    # Try extracting from specific HTML elements
    date_element = soup.find('span', {'class': 'published-date'}) or soup.find('time', {'datetime': True})
    if date_element:
        return date_element.get_text() or date_element['datetime']
    
    return None

# Function to extract just the date (YYYY-MM-DD) from the datetime string
def extract_date(datetime_str):
    try:
        # Parse the datetime string and format it to get just the date part
        date = datetime.fromisoformat(datetime_str).date()
        return str(date)
    except ValueError:
        # Handle cases where the datetime string is not in a standard format
        try:
            # Parse the datetime string without milliseconds and only return the date
            date = datetime.strptime(datetime_str.split('T')[0], '%Y-%m-%d').date()
            return str(date)
        except ValueError:
            try:
                # Parse the datetime string in the format '31 May 2024, 10:35 am'
                date = datetime.strptime(datetime_str.split(',')[0], '%d %B %Y').date()
                return str(date)
            except ValueError:
                return None

# Function to extract article date and cleaned content
def extract_article_details(html):
    # Extract publication date
    pub_date_raw = extract_pub_date(html)
    pub_date = extract_date(pub_date_raw) if pub_date_raw else 'Unknown'

    # Extract cleaned content
    soup = BeautifulSoup(html, 'html.parser')
    cleaned_content = ' '.join([text.strip() for text in soup.get_text(separator=' ', strip=True).split('\n')])

    return pub_date, cleaned_content

# Read URLs from master.csv
def read_urls_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    visited_urls = df['url'].tolist()
    return visited_urls

# Path to your master.csv file
csv_file_path = 'master.csv'

# Load state or read URLs from the CSV file
load_state()

# Read URLs from master.csv
visited_urls = read_urls_from_csv(csv_file_path)

# Filter out already processed URLs
urls_to_process = [url for url in visited_urls if url not in html_contents]

# Run the spiders
run_spiders(urls_to_process)

# Extract article details for each URL
data = []
for url, html in html_contents.items():
    article_date, cleaned_content = extract_article_details(html)
    data.append({
        'url': url,
        'raw_content': html,
        'cleaned_content': cleaned_content
    })

# Create a DataFrame
df_new = pd.DataFrame(data)

# Check if scrapy1.csv exists
if os.path.exists('scrapy1.csv'):
    # Load existing data
    df_existing = pd.read_csv('scrapy1.csv')

    # Merge new data with existing data, avoiding duplicates
    df = pd.concat([df_existing, df_new]).drop_duplicates(subset=['url']).reset_index(drop=True)
else:
    df = df_new

# Add a job_id column starting from 1 and incrementing
df['job_id'] = range(1, len(df) + 1)

# Save the DataFrame to CSV file scrapy1.csv
df.to_csv('scrapy1.csv', index=False)

# Check for existing chunked DataFrame files
existing_files = [f for f in os.listdir() if f.startswith('dataframe_chunk_') and f.endswith('.csv')]
processed_urls = set()

# Load processed URLs from existing chunk files
for file in existing_files:
    df_existing_chunk = pd.read_csv(file)
    processed_urls.update(df_existing_chunk['url'].tolist())

# Filter out already processed URLs from the DataFrame
df = df[~df['url'].isin(processed_urls)]

# Determine the starting chunk number
existing_chunk_numbers = [int(re.search(r'dataframe_chunk_(\d+)_', file).group(1)) for file in existing_files]
starting_chunk_number = max(existing_chunk_numbers, default=0) + 1

# Connect to SQLite database
conn = sqlite3.connect('scrapy_data.db')
cursor = conn.cursor()

# Dump the DataFrame in chunks of 400 rows
chunk_size = 400
num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size != 0 else 0)

for i in range(num_chunks):
    start_index = i * chunk_size
    end_index = start_index + chunk_size
    df_chunk = df.iloc[start_index:end_index]
    
    timestamp = datetime.now().strftime('%d%m%Y%H%M%S')
    table_name = f'dataframe_chunk_{starting_chunk_number + i}_date_{timestamp[:8]}_time_{timestamp[8:]}'
    print(table_name)
    
    # Save DataFrame chunk to SQL table
    df_chunk.to_sql(table_name, conn, if_exists='replace', index=False)
    



# Display the first few rows of the DataFrame
df.head()



/Users/sandipanray/anaconda3/lib/python3.10/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)


dataframe_chunk_1_date_03072024_time_162928
dataframe_chunk_2_date_03072024_time_162929
dataframe_chunk_3_date_03072024_time_162929


,url,raw_content,cleaned_content,job_id
0,https://www.republicworld.com/,"<!doctype html>\n<html lang=""en"" >\n\n<head>\n...","Latest News, News Today, Breaking News, India ...",1
1,https://www.republicworld.com/shorts/entertain...,"<!doctype html>\n<html lang=""en"" >\n\n<head>\n...","Varun Sharma, Rajkummar Rao Attend Sonakshi Si...",2
2,https://www.republicworld.com/automobile/auto-...,"<!doctype html>\n<html lang=""en"" >\n\n<head>\n...","Auto sales: M&M sold 40,022 SUVs, Bajaj Auto p...",3
3,https://www.republicworld.com/education,"<!doctype html>\n<html lang=""en"" >\n\n<head>\n...","Education news: Exams, Results, Class 10 & 12,...",4
4,https://www.republicworld.com/lifestyle/food/f...,"<!doctype html>\n<html lang=""en"" >\n\n<head>\n...",Five Essential Superfoods To Keep Working Wome...,5
